<h1>Shopping Store Management System</h1>
<h2>DA213, Python Programming Laboratory : Course Project</h2>
<h3>The Upside Down</h3>
<h4>M. Abhiram , 230150015 <br><br>
K. Ashmita , 230150014 <br><br>
B. Cherish , 230150007 <br></h4>

<h1>Importing the required Libraries</h1>

In [1]:
# For File Handling
import os

# For storing Date and Time in the standard format
from datetime import datetime

# For generating random order id
import random
import string

<h1>Defining the class Shop and implementing all the functionalities as the instance methods of the class</h1>

In [ ]:
# A class which contains all the information about the shop and also the functionalities
class Shop:

    Tax_Rate = 18   # Tax Rate, will be fixed

    # Constructor
    def __init__(self):     

        print("An object of the class Shop has successfully been created !!!\n")

        self.Discount_Rate=5   # Discount Rate, can be modified by the admin, for a given shop

        # Creating files if they don't exist
        self.initialize_files()

        # Data structures to store the data from the files
        self.Products={}
        self.Transactions=[]
        self.Admin_Transactions=[]

        # Loading the data from the files
        self.load_products_from_csv()
        self.load_transactions_from_csv()
        self.load_admin_transactions_from_csv()

     # Creating files if they do not exist
    def initialize_files(self):
        print("Initializing Files : ")
        # Products
        if not os.path.exists("Products.csv"):
            with open("Products.csv", "w") as file_create:
                file_create.write("Product Name,Price,Stock Quantity,Category,Brand\n")
                print("The file Products.csv has successfully been created")
        else:
            print("The file Products.csv already exists")

        # Transactions
        if not os.path.exists("Transactions.csv"):
            with open("Transactions.csv", "w") as file_create:
                file_create.write("Order ID,Product Name,Quantity,Original Price,Taxes,Discount,Final Price,Customer Name,Customer Phone No,Time of Purchase,Date of Purchase\n")
                print("The file Transactions.csv has successfully been created")
        else:
            print("The file Transactions.csv already exists")

        # Admi Transactions
        if not os.path.exists("Admin.csv"):
            with open("Admin.csv", "w") as file_create:
                file_create.write("Product Name,Quantity,Final Price,Time of Purchase,Date of Purchase\n")
                print("The file Admin.csv has successfully been created\n")
        else:
            print("The file Admin.csv already exists\n")

    # Loading the products information from the products file into the products dictionary
    def load_products_from_csv(self):
        self.Products={}
        try:
            with open("Products.csv", "r") as file_products:     # For error handling
                next(file_products)  # Skipping the 1t low
                for line in file_products:
                    line=line.strip()
                    if line:  # Skip empty lines
                        info=line.split(",")
                        if len(info) == 5:  # Ensure we have all fields
                            self.Products[info[0]]={'Price': float(info[1]),'Stock_Quantity': int(info[2]),'Category': info[3],'Brand': info[4]}
            print("Products from Products.csv file have successfully been loaded into the data structure\n")
        # Error Handling
        except FileNotFoundError:
            print("Products file not found. Creating a new one.\n")
            self.initialize_files()



    # Loading the transactions from the transactions file into the transactions list
    def load_transactions_from_csv(self):
        self.Transactions=[]
        try:
            with open("Transactions.csv", "r") as file_transactions:
                next(file_transactions)  # Skipping the 1st line
                for line in file_transactions:
                    line=line.strip()
                    if line:  # Skip empty lines
                        info=line.split(",")
                        if len(info) == 11:
                            self.Transactions.append({"Order_ID": info[0],"Product_Name": info[1], "Quantity": int(info[2]),"Original_Price": float(info[3]),"Taxes": float(info[4]),"Discount": float(info[5]),"Final_Price": float(info[6]),"Customer_Name": info[7],"Customer_Phone_No": info[8],"Time_of_Purchase": info[9],"Date_of_Purchase": info[10]})
            print("Products from Transactions.csv file have successfully been loaded into the data structure\n")
        # Error Handling
        except FileNotFoundError:
            print("Transactions file not found. Creating a new one.\n")
            self.initialize_files()


    # Loading the admin transactions from admin transactions file into the admin transactions in the same way as the above 2 processes
    def load_admin_transactions_from_csv(self):
        self.Admin_Transactions=[]
        try:
            with open("Admin.csv", "r") as admin_transactions:
                next(admin_transactions)  # Skip header
                for line in admin_transactions:
                    line=line.strip()
                    if line:  # Skip empty lines
                        info=line.split(",")
                        if len(info) == 5:  
                            self.Admin_Transactions.append({"Product_Name": info[0],"Quantity": int(info[1]),"Final_Price": float(info[2]),"Time_of_Purchase": info[3],"Date_of_Purchase": info[4]})
            print("Products from Admin.csv file have successfully been loaded into the data structure\n")
        except FileNotFoundError:
            print("Admin transactions file not found. Creating a new one.\n")
            self.initialize_files()

    def change_discount_rate(self,updated_rate):
        self.Discount_Rate = updated_rate
        print(f"New Discount Rate : {updated_rate}\n")

    # Generate a random 8-character order ID, when the buyer places the order
    def generate_order_id(self):
        order_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
        print(f"Order ID: {order_id}")
        return order_id

    # Getting the monthly profit/loss statement of the shop
    def get_monthly_financials(self):
        # A dictionary that stores all the profit/loss information for each month
        monthly_data={}

        # Iterating through all the transactions and appending all the transactions in the same month together
        for transaction in self.Transactions:
            try:
                # Extracting the date of a transaction
                date_str=transaction["Date_of_Purchase"]
                if len(date_str.split("-")[0]) == 2:  # If year is 2 digits
                    date_obj=datetime.strptime(date_str, "%y-%m-%d")
                else:  # If year is 4 digits
                    date_obj=datetime.strptime(date_str, "%Y-%m-%d")

                # Extracting only the month and year of the transaction
                year_month=date_obj.strftime("%Y-%m")  # Format as "YYYY-MM"

                # Initialising the key if the month does not exist already
                if year_month not in monthly_data:
                    monthly_data[year_month]={'sales': 0,'taxes_collected': 0,'discounts_given': 0,'expenses': 0,'net_profit': 0,'products_sold': {}}

                # Updating the dictionary accordingly
                monthly_data[year_month]['sales'] += transaction["Final_Price"]
                monthly_data[year_month]['taxes_collected'] += transaction["Taxes"]
                monthly_data[year_month]['discounts_given'] += transaction["Discount"]

                # Track products sold
                product=transaction["Product_Name"]
                qty=transaction["Quantity"]

                # Initialising the key if the product does not exist already
                if product not in monthly_data[year_month]['products_sold']:
                    monthly_data[year_month]['products_sold'][product]=0

                # Updating the quantity of a specific product sold in a given month
                monthly_data[year_month]['products_sold'][product] += qty

            # Error handling
            except Exception as e:
                print(f"Error processing transaction: {e}")
                continue

        # Iterating through all Admin Transactions and following the above process below, but this time we only update the 'expenses' key in the dictionary
        for transaction in self.Admin_Transactions:
            try:
                date_str=transaction["Date_of_Purchase"]
                if len(date_str.split("-")[0]) == 2:  
                    date_obj=datetime.strptime(date_str, "%y-%m-%d")
                else:  
                    date_obj=datetime.strptime(date_str, "%Y-%m-%d")

                year_month=date_obj.strftime("%Y-%m")  

                if year_month not in monthly_data:
                    monthly_data[year_month]={'sales': 0,'taxes_collected': 0,'discounts_given': 0,'expenses': 0,'net_profit': 0,'products_sold': {} }

                monthly_data[year_month]['expenses'] += transaction["Final_Price"]
            except Exception as e:
                print(f"Error processing admin transaction: {e}")
                continue

        # Calculate net profit
        for month in monthly_data:
            monthly_data[month]['net_profit']=monthly_data[month]['sales'] - monthly_data[month]['expenses']

        # Sort by year and month (newest first)
        sorted_months=sorted(monthly_data.keys(), reverse=True)
        return {month: monthly_data[month] for month in sorted_months}
    

   

   # Function to view all the products in shop
    def View_All_Products(self):
        print("List of all the available products\n")
        for product in self.Products:
            print(f"Product Name: {product}, Price: {self.Products[product]['Price']}, "
                  f"Stock Quantity: {self.Products[product]['Stock_Quantity']}, "
                  f"Category: {self.Products[product]['Category']}, "
                  f"Brand: {self.Products[product]['Brand']}")
            

    # Function to get information of a product by product name
    def Product_Info(self, Product_Name):
        check=False
        for product in self.Products:
            if product == Product_Name:
                check=True
                break
        if check:
            print(f"Product Name: {product}, Price: {self.Products[product]['Price']},Stock Quantity: {self.Products[product]['Stock_Quantity']}, "
                  f"Category: {self.Products[product]['Category']}, "
                  f"Brand: {self.Products[product]['Brand']}")
        else:
            print(f"{Product_Name} does not exist")


    # Restock the products into store
    def Restock(self, Product_Name, Quantity, Final_Price, Time_of_Purchase, Date_of_Purchase):
        check=False
        for product in self.Products:
            if product == Product_Name:
                check=True
                break

        if check:
            self.Products[Product_Name]['Stock_Quantity'] += Quantity
            self.Admin_Transactions.append({"Product_Name": Product_Name,"Quantity": int(Quantity),"Final_Price": float(Final_Price),"Time_of_Purchase": Time_of_Purchase,"Date_of_Purchase": Date_of_Purchase })

            # Update the Admin Transactions File
            with open("Admin.csv", "a") as file_transactions:
                file_transactions.write(f"\n{Product_Name},{Quantity},{Final_Price},{Time_of_Purchase},{Date_of_Purchase}")

            # Updating the Products File
            products_lines=["Product Name,Price,Stock Quantity,Category,Brand"]
            for product, details in self.Products.items():
                products_lines.append(f"{product},{details['Price']},{details['Stock_Quantity']},{details['Category']},{details['Brand']}")

            with open("Products.csv", "w") as file_products:
                file_products.write("\n".join(products_lines))

            print("Product Successfully Restocked")
            print("Details:")
            print(f"Product Name: {Product_Name}")
            print(f"Quantity Added: {Quantity}")
            print(f"Expenditure: {Final_Price}")
            print(f"Time of Purchase: {Time_of_Purchase}")
            print(f"Date of Purchase: {Date_of_Purchase}\n")
        else:
            print(f"{Product_Name} does not exist\n")


    # Similar to restock but this adds a new product into Inventory
    def Buy_New_Products(self, Product_Name, Category, Brand, Price, Quantity, Final_Price, Time_of_Purchase, Date_of_Purchase):
        check=False
        for product in self.Products:
            if product == Product_Name:
                check=True
                break

        if check:
            print(f"{Product_Name} already exists")
        else:
            self.Products[Product_Name]={'Price': Price,'Stock_Quantity': Quantity,'Category': Category,'Brand': Brand}
            self.Admin_Transactions.append({"Product_Name": Product_Name,"Quantity": Quantity,"Final_Price": Final_Price,"Time_of_Purchase": Time_of_Purchase,"Date_of_Purchase": Date_of_Purchase})

            # Update the Admin Transactions File
            with open("Admin.csv", "a") as file_transactions:
                file_transactions.write(f"\n{Product_Name},{Quantity},{Final_Price},{Time_of_Purchase},{Date_of_Purchase}")

            # Updating the Products File
            with open("Products.csv", "a") as file_products:
                file_products.write(f"\n{Product_Name},{Price},{Quantity},{Category},{Brand}")

            print("New Product Successfully Added")
            print("Details:")
            print(f"Product Name: {Product_Name}")
            print(f"Brand: {Brand}")
            print(f"Category: {Category}")
            print(f"Quantity: {Quantity}")
            print(f"Minimum Retail Price: {Price}")
            print(f"Expenditure: {Final_Price}")
            print(f"Time of Purchase: {Time_of_Purchase}")
            print(f"Date of Purchase: {Date_of_Purchase}\n")



    # Function to get zero stock alert or may be low stock alert if the quantity of products is less than the threshold of 10 items
    def Zero_Stock_Alert(self):
        zero_stock_products=[]
        print("The following quantities have zero or very few amount of stock left:")
        for product in self.Products:
            if self.Products[product]['Stock_Quantity'] < 10:
                print(product)
                zero_stock_products.append((product,self.Products[product]['Stock_Quantity']))
        return zero_stock_products

    # Function to implement feature of Customer Buying products
    def buy_products(self, product_name, quantity, customer_name, customer_phone, time_str, date_str,order_id):
        if product_name not in self.Products:
            print(f"{product_name} does not exist")
            return False

        if self.Products[product_name]['Stock_Quantity'] < quantity:
            print(f"There isn't enough stock left for your purchase of {product_name}")
            return False

        
        original_price=self.Products[product_name]['Price'] * quantity
        taxes=(self.Tax_Rate / 100) * original_price
        discount=(self.Discount_Rate / 100) * original_price
        final_price=original_price + taxes - discount

        # Updating the stock quantity
        self.Products[product_name]['Stock_Quantity'] -= quantity

        # Updating the transactions list
        self.Transactions.append({"Order_ID": order_id,"Product_Name": product_name,"Quantity": quantity,"Original_Price": original_price,"Taxes": taxes,"Discount": discount,"Final_Price": final_price,"Customer_Name": customer_name,"Customer_Phone_No": customer_phone,"Time_of_Purchase": time_str,"Date_of_Purchase": date_str})

        # Updating the  Products file
        products_lines=["Product Name,Price,Stock Quantity,Category,Brand"]
        for product, details in self.Products.items():
            products_lines.append(f"{product},{details['Price']},{details['Stock_Quantity']},{details['Category']},{details['Brand']}")

        with open("Products.csv", "w") as file_products:
            file_products.write("\n".join(products_lines))

        # Update the Transactions file
        with open("Transactions.csv", "a") as file_transactions:
            file_transactions.write(
                f"\n{order_id},{product_name},{quantity},{original_price},{taxes},{discount},{final_price},"
                f"{customer_name},{customer_phone},{time_str},{date_str}"
            )
        
        print("Product Successfully Purchased")
        print("Details:")
        print(f"Customer Name: {customer_name}")
        print(f"Customer Phone No: {customer_phone}")
        print(f"Product Name: {product_name}")
        print(f"Brand: {self.Products[product_name]['Brand']}")
        print(f"Category: {self.Products[product_name]['Category']}")
        print(f"Quantity: {quantity}")
        print(f"Sub Total: {original_price}")
        print(f"Discount: {discount}")
        print(f"Taxes: {taxes}")
        print(f"Total: {final_price}")
        print(f"Time of Purchase: {time_str}")
        print(f"Date of Purchase: {date_str}\n")

        return True

<h1>Importing TkInter for Graphical User Interface</h1>

In [3]:
# TkInter Library for Graphical User Interface
import tkinter as tk
from tkinter import ttk, messagebox
from tkinter.font import Font

<h1>Designing the Graphical User Interface, along with Integrating the Backend part of the Code</h1>

In [4]:
# Class which contains all the methods for the GUI
class ShoppingApp:

    def __init__(self, root):
        self.root=root
        self.root.title("Shopping Store Management System")
        self.root.geometry("1200x800")
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)  # Handle window close

        # Initialize shop
        self.shop=Shop()
        self.cart={}
        self.admin_logged_in=False

        # Create fonts
        self.title_font=Font(family="Helvetica", size=16, weight="bold")
        self.button_font=Font(family="Helvetica", size=12)
        self.product_font=Font(family="Helvetica", size=11)

        # Configure styles
        self.configure_styles()

        # Create main interface
        self.create_interface()

    # A dashboard after the admin login
    def create_dashboard_tab(self, parent):
        # Main frame
        dashboard_frame=ttk.Frame(parent, padding=15)
        dashboard_frame.pack(expand=True, fill="both")

        # Title
        ttk.Label(dashboard_frame,
                text="Monthly Sales Report",
                font=("Helvetica", 16, "bold")
            ).pack(pady=(0, 15))

        # Month selection
        month_frame=ttk.Frame(dashboard_frame)
        month_frame.pack(fill="x", pady=(0, 15))

        ttk.Label(month_frame, text="Select Month:").pack(side="left", padx=(0, 10))

        # Get available months
        financial_data=self.shop.get_monthly_financials()
        months=list(financial_data.keys())

        if not months:
            ttk.Label(dashboard_frame,
                text="No sales data available yet",
                font=("Helvetica", 12)
            ).pack(expand=True)
            return

        month_display=[]
        for ym in months:
            year, month=ym.split('-')
            month_name=datetime.strptime(month, "%m").strftime("%B")  # "04" → "April"
            month_display.append(f"{month_name} {year}")  # "April 2024"

        self.selected_month=tk.StringVar()
        month_dropdown=ttk.Combobox(
            month_frame,
            textvariable=self.selected_month,
            values=month_display,  # Show "April 2024" instead of "24-04"
            state="readonly"
        )
        month_dropdown.pack(side="left")
        month_dropdown.bind("<<ComboboxSelected>>", lambda e: self.update_monthly_report())

        # Report frame
        self.report_frame=ttk.LabelFrame(
            dashboard_frame,
            text="Monthly Report",
            padding=15
        )
        self.report_frame.pack(expand=True, fill="both")

        # Initial report
        self.update_monthly_report()

    def update_monthly_report(self):
        # Clear previous report
        for widget in self.report_frame.winfo_children():
            widget.destroy()

        # Get data for selected month
        month_display=self.selected_month.get()
        if not month_display:
            ttk.Label( self.report_frame,
                text="Please select a month",
                font=("Helvetica", 12)
            ).pack(expand=True)
            return

        # Convert display format ("April 2024") back to key format ("2024-04")
        month_name, year=month_display.split()
        month_num=datetime.strptime(month_name, "%B").strftime("%m")
        month_key=f"{year}-{month_num}"

        ### Retrieving the monthly statistics from the backend
        financial_data=self.shop.get_monthly_financials()
        data=financial_data.get(month_key, {})

        if not data:
            ttk.Label( self.report_frame,
                text=f"No data available for {month_display}",
                font=("Helvetica", 12)
            ).pack(expand=True)
            return

        # Create report content
        report_content=[
            ("Total Sales", f"₹{data['sales']:,.2f}"),
            ("Total Expenses", f"₹{data['expenses']:,.2f}"),
            ("", ""),  # Spacer
            ("Taxes Collected", f"₹{data['taxes_collected']:,.2f}"),
            ("Discounts Given", f"₹{data['discounts_given']:,.2f}"),
            ("", ""),  # Spacer
            ("Net Profit", f"₹{data['net_profit']:,.2f}", "green" if data['net_profit'] >= 0 else "red")
        ]

        for item in report_content:
            if item[0] == "":  # Spacer
                ttk.Separator(self.report_frame).pack(fill="x", pady=5)
                continue

            row_frame=ttk.Frame(self.report_frame)
            row_frame.pack(fill="x", pady=3)

            ttk.Label(row_frame,
                text=item[0],
                font=("Helvetica", 11),
                width=20,
                anchor="w"
            ).pack(side="left")

            fg=item[2] if len(item) > 2 else "black"
            ttk.Label(row_frame,
                text=item[1],
                font=("Helvetica", 11, "bold"),
                foreground=fg,
                anchor="e"
            ).pack(side="right")

        # Top products
        ttk.Label(self.report_frame,
            text="\nTop Selling Products:",
            font=("Helvetica", 11)
        ).pack(anchor="w", pady=(10, 5))

        top_products=sorted(data['products_sold'].items(),
            key=lambda x: x[1],
            reverse=True
        )[:3]  # Show top 3 products

        for product, qty in top_products:
            product_frame=ttk.Frame(self.report_frame)
            product_frame.pack(fill="x", pady=2)

            ttk.Label(product_frame,
                text=f"• {product}",
                font=("Helvetica", 10),
                width=30,
                anchor="w"
            ).pack(side="left")

            ttk.Label(product_frame,
                text=f"{qty} units sold",
                font=("Helvetica", 10),
                foreground="#555"
            ).pack(side="right")

    #For displaying a block of metric card like monthly sales 25000 etc..
    def create_metric_card(self, parent, title, value, row, column, color="black"):
        card=ttk.Frame(parent, style="Card.TFrame", padding=10)
        card.grid(row=row, column=column, padx=5, pady=5, sticky="nsew")

        parent.columnconfigure(column, weight=1)
        parent.rowconfigure(row, weight=1)

        ttk.Label(card,
            text=title,
            font=("Helvetica", 9),
            foreground="#666"
        ).pack(anchor="w")

        ttk.Label(card,
            text=value,
            font=("Helvetica", 14, "bold"),
            foreground=color
        ).pack(anchor="w", pady=(5, 0))
        
    def refresh_dashboard(self, parent):
        for widget in parent.winfo_children():
            widget.destroy()
        self.create_dashboard_tab(parent)

# Function which styles widgets color,font,view etc...
    def configure_styles(self):

        style=ttk.Style()
        style.theme_use('clam')

        style.configure("Treeview", font=self.product_font, rowheight=25)
        style.configure("Treeview.Heading", font=self.button_font)

        style.configure("Accent.TButton",
                      font=self.button_font,
                      foreground="white",
                      background="#2E8B57",
                      padding=6)

        style.map("Accent.TButton",
                background=[("active", "#3CB371"), ("disabled", "#C0C0C0")])

        style.configure("Admin.TButton",
                      font=self.button_font,
                      foreground="white",
                      background="#4682B4",  # SteelBlue color
                      padding=6)

        style.map("Admin.TButton",
                background=[("active", "#5F9EA0"), ("disabled", "#C0C0C0")])

        style.configure("Warning.TButton",
                      font=self.button_font,
                      foreground="white",
                      background="#CD5C5C",
                      padding=6)

        style.map("Warning.TButton",
                background=[("active", "#DC143C"), ("disabled", "#C0C0C0")])
        style.configure("Card.TFrame", background="#f5f5f5", relief="solid", borderwidth=1)


   # Proper closing of application
    def on_close(self):
        """Proper cleanup when closing the application"""
        self.root.quit()
        self.root.destroy()

    # Initial interface of Application
    def create_interface(self):
        # Main frame
        self.main_frame=ttk.Frame(self.root)
        self.main_frame.pack(expand=True, fill="both", padx=10, pady=10)

        # Admin button at top right
        self.admin_btn=ttk.Button(
            self.main_frame,
            text="Admin Login",
            command=self.admin_login,
            style="Admin.TButton"
        )
        self.admin_btn.pack(anchor="ne", padx=5, pady=5)

        # Product display
        self.create_product_display()

        # Bottom controls
        self.bottom_frame=ttk.Frame(self.main_frame)
        self.bottom_frame.pack(side="bottom", fill="x", pady=(10, 0))

        self.view_cart_btn=ttk.Button(
            self.bottom_frame,
            text="View Cart",
            command=self.view_cart_window,
            style="Accent.TButton"
        )
        self.view_cart_btn.pack(side="right", padx=5)

        self.exit_btn=ttk.Button(
            self.bottom_frame,
            text="Exit",
            command=self.root.quit,
            style="Warning.TButton"
        )
        self.exit_btn.pack(side="right", padx=5)


    # Admin login feature with username is upsidedown and password is giveup
    def admin_login(self):
        if self.admin_logged_in:
            self.show_admin_panel()
            return

        login_window=tk.Toplevel(self.root)
        login_window.title("Admin Login")
        login_window.geometry("300x200")

        ttk.Label(login_window, text="Admin Login", font=self.title_font).pack(pady=10)

        ttk.Label(login_window, text="Username:").pack()
        self.username_entry=ttk.Entry(login_window)
        self.username_entry.pack(pady=5)

        ttk.Label(login_window, text="Password:").pack()
        self.password_entry=ttk.Entry(login_window, show="*")
        self.password_entry.pack(pady=5)

        def verify():
            if self.username_entry.get() == "upsidedown" and self.password_entry.get() == "giveup":
                self.admin_logged_in=True
                print("Admin Successfully Logged In\n")
                login_window.destroy()
                self.admin_btn.config(text="Admin Panel")
                self.show_admin_panel()
            else:
                messagebox.showerror("Error", "Invalid credentials")
                print("Admin Authentication Failed\n")

        ttk.Button(
            login_window,
            text="Login",
            command=verify,
            style="Accent.TButton"
        ).pack(pady=10)

    #Top Nav bar with tabbed interface
    def show_admin_panel(self):
        admin_window=tk.Toplevel(self.root)
        admin_window.title("Admin Panel")
        admin_window.geometry("1000x700")

        # Notebook for different admin sections
        notebook=ttk.Notebook(admin_window)
        notebook.pack(expand=True, fill="both", padx=10, pady=10)

        # Dashboard Tab (new)
        dashboard_frame=ttk.Frame(notebook)
        notebook.add(dashboard_frame, text="Dashboard")
        self.create_dashboard_tab(dashboard_frame)

        # Restock Tab
        restock_frame=ttk.Frame(notebook)
        notebook.add(restock_frame, text="Restock Products")
        self.create_restock_tab(restock_frame)

        # Add Product Tab
        add_product_frame=ttk.Frame(notebook)
        notebook.add(add_product_frame, text="Add New Product")
        self.create_add_product_tab(add_product_frame)

        # View Inventory Tab
        inventory_frame=ttk.Frame(notebook)
        notebook.add(inventory_frame, text="View Inventory")
        self.create_inventory_tab(inventory_frame)

        # View Customer Transactions Tab
        transactions_frame=ttk.Frame(notebook)
        notebook.add(transactions_frame, text="Customer Transactions")
        self.create_transactions_tab(transactions_frame)

        # View Admin Transactions Tab
        admin_transactions_frame=ttk.Frame(notebook)
        notebook.add(admin_transactions_frame, text="Admin Transactions")
        self.create_admin_transactions_tab(admin_transactions_frame)

        # Update Discount Tab
        update_discount_frame=ttk.Frame(notebook)
        notebook.add(update_discount_frame, text="Update Discount")
        self.create_update_discount_tab(update_discount_frame)

    # creating restock tab
    def create_restock_tab(self, parent):

        # Product selection
        ttk.Label(parent, text="Select Product to Restock:").pack(pady=5)

        self.restock_product_var=tk.StringVar()
        product_names=list(self.shop.Products.keys())
        if not product_names:
            product_names=["No products available"]
            self.restock_product_var.set(product_names[0])
        else:
            self.restock_product_var.set(product_names[0])

        self.product_combobox=ttk.Combobox(
            parent,
            textvariable=self.restock_product_var,
            values=product_names,
            state="readonly"
        )
        self.product_combobox.pack(pady=5)

        # Quantity
        ttk.Label(parent, text="Quantity to Add:").pack(pady=5)
        self.restock_qty_var=tk.IntVar(value=1)
        ttk.Spinbox(
            parent,
            from_=1,
            to=1000,
            textvariable=self.restock_qty_var,
            width=5
        ).pack(pady=5)


        # Price
        ttk.Label(parent, text="Total Cost for This Restock:").pack(pady=5)
        self.restock_price_var=tk.DoubleVar(value=0.0)
        ttk.Entry(
            parent,
            textvariable=self.restock_price_var
        ).pack(pady=5)


        # Restock button
        ttk.Button(
            parent,
            text="Restock Product",
            command=self.process_restock,
            style="Accent.TButton"
        ).pack(pady=10)


        # Zero stock alert
        ttk.Button(
            parent,
            text="Check Low Stock Items",
            command=self.show_low_stock_alert,
            style="Warning.TButton"
        ).pack(pady=10)


    #processing restock basically calling restock function with required parameters
    def process_restock(self):
        product_name=self.restock_product_var.get()
        quantity=self.restock_qty_var.get()
        price=self.restock_price_var.get()

        if product_name == "No products available":
            messagebox.showwarning("No Products", "There are no products to restock")
            return

        if quantity<=0:
            messagebox.showwarning("Invalid Quantity", "Quantity must be at least 1")
            return

        if price < 0:
            messagebox.showwarning("Invalid Price", "Price cannot be negative")
            return

        now=datetime.now()
        time_str=now.strftime("%H:%M:%S")
        date_str=now.strftime("%Y-%m-%d")

        try:
            self.shop.Restock(Product_Name=product_name,Quantity=quantity,Final_Price=price,Time_of_Purchase=time_str,Date_of_Purchase=date_str)
            messagebox.showinfo("Success", f"Successfully restocked {quantity} {product_name}(s)")
            self.populate_products()  # Refresh product list
        except Exception as e:
            messagebox.showerror("Error", f"Failed to restock product: {str(e)}")

    # Showing low stock alert for products whose quantity is less than 10
    def show_low_stock_alert(self):
        ### Retrieving the information from the backend
        low_stock_list = self.shop.Zero_Stock_Alert()
        low_stock = []
        for product,quantity in low_stock_list:
            low_stock.append(f"{product} - Only {quantity} left")

        if not low_stock:
            messagebox.showinfo("Stock Status", "All products have sufficient stock")
        else:
            alert_window=tk.Toplevel(self.root)
            alert_window.title("Low Stock Alert")

            ttk.Label(alert_window, text="Low Stock Items:", font=self.title_font).pack(pady=10)

            for item in low_stock:
                ttk.Label(alert_window, text=item).pack(anchor="w", padx=10)

    # Add product to inventory tab
    def create_add_product_tab(self, parent):
        # Form fields
        fields=[("Product Name:", "entry"),("Price:", "entry"),("Initial Stock Quantity:", "entry"),("Category:", "entry"),("Brand:", "entry")]

        self.add_product_vars={}

        for i, (label_text, field_type) in enumerate(fields):
            frame=ttk.Frame(parent)
            frame.pack(fill="x", padx=10, pady=5)

            ttk.Label(frame, text=label_text).pack(side="left")

            if field_type == "entry":
                var=tk.StringVar()
                entry=ttk.Entry(frame, textvariable=var)
                entry.pack(side="right", expand=True, fill="x", padx=10)
                self.add_product_vars[label_text.lower().replace(":", "").replace(" ", "_")]=var

        # Add product button
        ttk.Button(parent,
            text="Add New Product",
            command=self.process_add_product,
            style="Accent.TButton"
        ).pack(pady=20)

    def process_add_product(self):
        try:
            product_name=self.add_product_vars["product_name"].get()
            price=float(self.add_product_vars["price"].get())
            stock=int(self.add_product_vars["initial_stock_quantity"].get())
            category=self.add_product_vars["category"].get()
            brand=self.add_product_vars["brand"].get()

            if not product_name:
                messagebox.showwarning("Invalid Input", "Product name cannot be empty")
                return

            if price<=0:
                messagebox.showwarning("Invalid Input", "Price must be positive")
                return

            if stock < 0:
                messagebox.showwarning("Invalid Input", "Stock cannot be negative")
                return

            now=datetime.now()
            time_str=now.strftime("%H:%M:%S")
            date_str=now.strftime("%Y-%m-%d")

            # Calculate final price (cost to shop)
            # In a real system, this would be different from retail price
            final_price=price * stock * 0.8  # Assuming 20% margin

            ### Calling the function in the backend
            self.shop.Buy_New_Products(
                Product_Name=product_name,
                Category=category,
                Brand=brand,
                Price=price,
                Quantity=stock,
                Final_Price=final_price,
                Time_of_Purchase=time_str,
                Date_of_Purchase=date_str
            )

            messagebox.showinfo("Success", f"Successfully added new product: {product_name}")

            # Clear form
            for var in self.add_product_vars.values():
                var.set("")

            # Refresh product list
            self.populate_products()

        except ValueError:
            messagebox.showerror("Error", "Please enter valid numbers for price and quantity")

    # Updating Discount
    def create_update_discount_tab(self,parent):
        # Main frame
        frame=ttk.Frame(parent, padding=15)
        frame.pack(expand=True, fill="both")

        # Title
        ttk.Label(frame,
                text="Update Discount",
                font=("Helvetica", 16, "bold")
            ).pack(pady=(0, 15))
        
        self.current_discount_var = tk.StringVar()
        self.current_discount_var.set(f"Current Discount : {self.shop.Discount_Rate}%")  

        ttk.Label(frame,
                textvariable=self.current_discount_var,
                font=("Helvetica", 13,"bold"),  # Simpler font
                anchor="w"               # Align text to left
            ).pack(fill="x", padx=10, pady=5)
        
        # Form fields
        fields=[("New Discount:", "entry")]

        self.update_discount_vars={}

        for i, (label_text, field_type) in enumerate(fields):
            frame_1=ttk.Frame(frame)
            frame_1.pack(fill="x", padx=10, pady=10)

            ttk.Label(frame_1, text=label_text).pack(side="left")

            if field_type == "entry":
                var=tk.StringVar()
                entry=ttk.Entry(frame_1, textvariable=var,width=12)
                entry.pack(side="left", padx=10)
                self.update_discount_vars[label_text.lower().replace(":", "").replace(" ", "_")]=var

        # Update Discount button
        button_frame = ttk.Frame(frame)
        button_frame.pack(pady=10, fill="x")
        ttk.Button(button_frame,
            text="Update",
            command=self.process_update_discount,
            style="Accent.TButton"
        ).pack(padx=115,side="left")
        
    def process_update_discount(self):
        updated_discount =float(self.update_discount_vars["new_discount"].get())
        if(updated_discount>100 or updated_discount<0):
            messagebox.showwarning("Invalid Input", "Discount has to be a number between 0 and 100")
            return
        self.shop.change_discount_rate(updated_discount)
        messagebox.showinfo("Success", f"Successfully Updated the Discount to: {updated_discount}%")
        self.current_discount_var.set(f"Current Discount : {self.shop.Discount_Rate}%")

        # Clear form
        for var in self.update_discount_vars.values():
            var.set("")


    # Function to display all items in the inventory in a tab
    def create_inventory_tab(self, parent):
        # Treeview to display all products
        tree_scroll=ttk.Scrollbar(parent)
        tree_scroll.pack(side="right", fill="y")

        self.inventory_tree=ttk.Treeview(parent,
            columns=("Name", "Price", "Stock", "Category", "Brand"),
            show="headings",
            yscrollcommand=tree_scroll.set,
            selectmode="browse"
        )
        tree_scroll.config(command=self.inventory_tree.yview)

        # Configure columns
        columns={
            "Name": {"text": "Product Name", "width": 200, "anchor": "w"},
            "Price": {"text": "Price", "width": 100, "anchor": "e"},
            "Stock": {"text": "Stock", "width": 80, "anchor": "center"},
            "Category": {"text": "Category", "width": 150, "anchor": "w"},
            "Brand": {"text": "Brand", "width": 150, "anchor": "w"}
        }

        for col, config in columns.items():
            self.inventory_tree.heading(col, text=config["text"])
            self.inventory_tree.column(col, width=config["width"], anchor=config["anchor"])

        self.inventory_tree.pack(expand=True, fill="both", padx=5, pady=5)

        # Populate the tree
        self.populate_inventory_tree()

        # Refresh button
        ttk.Button(
            parent,
            text="Refresh Inventory",
            command=self.populate_inventory_tree,
            style="Accent.TButton"
        ).pack(pady=10)

    # Filling the columns in Tree(basically a table) with the products and details
    def populate_inventory_tree(self):
        # Clear existing items
        for item in self.inventory_tree.get_children():
            self.inventory_tree.delete(item)

        # Reload products from CSV
        print("Reloading the Page",end=" : ")
        self.shop.load_products_from_csv()

        # Add products to tree
        for product_name, details in self.shop.Products.items():
            self.inventory_tree.insert("",
                                       "end",
                                       values=(product_name,
                                        f"₹{details['Price']:.2f}",
                                        details['Stock_Quantity'],
                                        details['Category'],
                                        details['Brand']
                                         ))

    # A tab to display all the customer transactions
    def create_transactions_tab(self, parent):
        # Treeview to display customer transactions
        tree_scroll=ttk.Scrollbar(parent)
        tree_scroll.pack(side="right", fill="y")

        self.transactions_tree=ttk.Treeview(
            parent,
            columns=("OrderID", "Product", "Qty", "Price", "Tax", "Discount", "Total", "Customer", "Phone", "Time", "Date"),
            show="headings",
            yscrollcommand=tree_scroll.set
        )
        tree_scroll.config(command=self.transactions_tree.yview)

        # Configure columns
        columns={
            "OrderID": {"text": "Order ID", "width": 100, "anchor": "center"},
            "Product": {"text": "Product", "width": 150, "anchor": "w"},
            "Qty": {"text": "Qty", "width": 50, "anchor": "center"},
            "Price": {"text": "Price", "width": 80, "anchor": "e"},
            "Tax": {"text": "Tax", "width": 80, "anchor": "e"},
            "Discount": {"text": "Discount", "width": 80, "anchor": "e"},
            "Total": {"text": "Total", "width": 90, "anchor": "e"},
            "Customer": {"text": "Customer", "width": 120, "anchor": "w"},
            "Phone": {"text": "Phone", "width": 100, "anchor": "w"},
            "Time": {"text": "Time", "width": 80, "anchor": "center"},
            "Date": {"text": "Date", "width": 90, "anchor": "center"}
        }

        for col, config in columns.items():
            self.transactions_tree.heading(col, text=config["text"])
            self.transactions_tree.column(col, width=config["width"], anchor=config["anchor"])

        self.transactions_tree.pack(expand=True, fill="both", padx=5, pady=5)

        # Populate the tree
        self.populate_transactions_tree()

        # Refresh button
        ttk.Button(
            parent,
            text="Refresh Transactions",
            command=self.populate_transactions_tree,
            style="Accent.TButton"
        ).pack(pady=10)

    # Filling the customer transactions table with the data from csv file
    def populate_transactions_tree(self):
        # Clear existing items
        for item in self.transactions_tree.get_children():
            self.transactions_tree.delete(item)

        # Reload transactions from CSV
        print("Reloading the Page",end=" : ")
        self.shop.load_transactions_from_csv()

        # Add transactions to tree
        for transaction in self.shop.Transactions:
            self.transactions_tree.insert("", "end", values=(
                transaction["Order_ID"],
                transaction["Product_Name"],
                transaction["Quantity"],
                f"₹{transaction['Original_Price']:.2f}",
                f"₹{transaction['Taxes']:.2f}",
                f"₹{transaction['Discount']:.2f}",
                f"₹{transaction['Final_Price']:.2f}",
                transaction["Customer_Name"],
                transaction["Customer_Phone_No"],
                transaction["Time_of_Purchase"],
                transaction["Date_of_Purchase"]
            ))

    # A tab for displaying admin transactions
    def create_admin_transactions_tab(self, parent):
        # Treeview to display admin transactions (restocks)
        tree_scroll=ttk.Scrollbar(parent)
        tree_scroll.pack(side="right", fill="y")

        self.admin_transactions_tree=ttk.Treeview(
            parent,
            columns=("Product", "Qty", "Cost", "Time", "Date"),
            show="headings",
            yscrollcommand=tree_scroll.set
        )
        tree_scroll.config(command=self.admin_transactions_tree.yview)

        # Configure columns
        columns={
            "Product": {"text": "Product", "width": 200, "anchor": "w"},
            "Qty": {"text": "Qty", "width": 80, "anchor": "center"},
            "Cost": {"text": "Total Cost", "width": 100, "anchor": "e"},
            "Time": {"text": "Time", "width": 80, "anchor": "center"},
            "Date": {"text": "Date", "width": 90, "anchor": "center"}
        }

        for col, config in columns.items():
            self.admin_transactions_tree.heading(col, text=config["text"])
            self.admin_transactions_tree.column(col, width=config["width"], anchor=config["anchor"])

        self.admin_transactions_tree.pack(expand=True, fill="both", padx=5, pady=5)

        # Populate the tree
        self.populate_admin_transactions_tree()

        # Refresh button
        ttk.Button(
            parent,
            text="Refresh Transactions",
            command=self.populate_admin_transactions_tree,
            style="Accent.TButton"
        ).pack(pady=10)

    # Filling the Admi transactions table
    def populate_admin_transactions_tree(self):
        # Clear existing items
        for item in self.admin_transactions_tree.get_children():
            self.admin_transactions_tree.delete(item)

        # Reload admin transactions from CSV
        print("Reloading the Page",end=" : ")
        self.shop.load_admin_transactions_from_csv()

        # Add transactions to tree
        for transaction in self.shop.Admin_Transactions:
            self.admin_transactions_tree.insert("", "end", values=(
                transaction["Product_Name"],
                transaction["Quantity"],
                f"₹{transaction['Final_Price']:.2f}",
                transaction["Time_of_Purchase"],
                transaction["Date_of_Purchase"]
            ))

    # Displaying products in the interface
    def create_product_display(self):
        # Product frame
        self.product_frame=ttk.LabelFrame(self.main_frame, text="Available Products", padding=10)
        self.product_frame.pack(expand=True, fill="both")

        # Treeview with scrollbars
        self.tree_scroll=ttk.Scrollbar(self.product_frame)
        self.tree_scroll.pack(side="right", fill="y")

        self.products_tree=ttk.Treeview(
            self.product_frame,
            columns=("Name", "Brand", "Category", "Price"),
            show="headings",
            yscrollcommand=self.tree_scroll.set,
            selectmode="browse"
        )
        self.tree_scroll.config(command=self.products_tree.yview)

        # Configure columns - all centered
        columns={
            "Name": {"text": "Product Name", "width": 250, "anchor": "center"},
            "Brand": {"text": "Brand", "width": 150, "anchor": "center"},
            "Category": {"text": "Category", "width": 150, "anchor": "center"},
            "Price": {"text": "Price (₹)", "width": 100, "anchor": "center"}
        }

        for col, config in columns.items():
            self.products_tree.heading(col, text=config["text"])
            self.products_tree.column(col, width=config["width"], anchor=config["anchor"])

        self.products_tree.pack(expand=True, fill="both")
        self.populate_products()

        # Quantity controls
        self.controls_frame=ttk.Frame(self.product_frame)
        self.controls_frame.pack(fill="x", pady=(10, 0))

        ttk.Label(self.controls_frame, text="Quantity:").pack(side="left", padx=(0, 5))

        self.quantity_var=tk.IntVar(value=1)
        self.quantity_spin=ttk.Spinbox(
            self.controls_frame,
            from_=1,
            to=1,  # Initial value, will be updated when product is selected
            textvariable=self.quantity_var,
            width=5
        )
        self.quantity_spin.pack(side="left", padx=(0, 10))

        # Bind selection event to update spinbox max value
        self.products_tree.bind("<<TreeviewSelect>>", self.update_spinbox_range)

        self.add_to_cart_btn=ttk.Button(
            self.controls_frame,
            text="Add to Cart",
            command=self.add_to_cart,
            style="Accent.TButton"
        )
        self.add_to_cart_btn.pack(side="left")

    # spinbox for selecting no. of items of a product to buy
    def update_spinbox_range(self, event):
        """Update spinbox range based on selected product's stock"""
        selected_item=self.products_tree.focus()
        if selected_item:
            item_data=self.products_tree.item(selected_item)
            product_name=item_data['values'][0]
            current_stock=self.shop.Products[product_name]['Stock_Quantity']

            # Update spinbox range (minimum 1, maximum current stock)
            self.quantity_spin.config(from_=1, to=max(1, current_stock))

            # Get current quantity value
            current_val=self.quantity_var.get()

            # If current value exceeds new max, set to max
            if current_val > current_stock:
                self.quantity_var.set(max(1, current_stock))
            elif current_val < 1:  # Ensure minimum is 1
                self.quantity_var.set(1)

    # Filling the products table from the file
    def populate_products(self):
        for item in self.products_tree.get_children():
            self.products_tree.delete(item)

        # Reload products from CSV to ensure we have latest data
        print("Reloading the Page",end=" : ")
        self.shop.load_products_from_csv()

        for product_name, details in self.shop.Products.items():
            self.products_tree.insert("", "end", values=(
                product_name,
                details['Brand'],
                details['Category'],
                f"₹{details['Price']:.2f}"
            ))

    # Adding items to cart
    def add_to_cart(self):
        selected_item=self.products_tree.focus()
        if not selected_item:
            messagebox.showwarning("No Selection", "Please select a product first")
            return

        item_data=self.products_tree.item(selected_item)
        product_name=item_data['values'][0]
        current_stock=self.shop.Products[product_name]['Stock_Quantity']
        quantity=self.quantity_var.get()

        if quantity<=0:
            messagebox.showwarning("Invalid Quantity", "Quantity must be at least 1")
            return

        if quantity>current_stock:
            messagebox.showwarning("Insufficient Stock",
                                f"Only {current_stock} items available in stock")
            return

        # Add to cart or update quantity
        if product_name in self.cart:
            # If product already in cart, add the new quantity to existing quantity
            new_quantity=self.cart[product_name]+quantity
            if new_quantity>current_stock:
                messagebox.showwarning("Insufficient Stock",
                                    f"Can't add more than available stock ({current_stock})")
                return
            self.cart[product_name]=new_quantity
        else:
            # If product not in cart, add it with the selected quantity
            self.cart[product_name]=quantity

        messagebox.showinfo("Success", f"{quantity} {product_name}(s) added to cart")
        self.quantity_var.set(1)  # Reset quantity to 1 after adding to cart

    # window to view items in a cart
    def view_cart_window(self):
        if not self.cart:
            messagebox.showinfo("Empty Cart", "Your cart is empty")
            return

        cart_window=tk.Toplevel(self.root)
        cart_window.title("Your Shopping Cart")
        cart_window.geometry("800x600")

        cart_frame=ttk.Frame(cart_window, padding=10)
        cart_frame.pack(expand=True, fill="both")

        ttk.Label(cart_frame, text="Your Shopping Cart", font=self.title_font).pack(pady=(0, 10))

        # Cart treeview
        cart_tree_scroll=ttk.Scrollbar(cart_frame)
        cart_tree_scroll.pack(side="right", fill="y")

        self.cart_tree=ttk.Treeview(
            cart_frame,
            columns=("Name", "Price", "Quantity", "Subtotal"),
            show="headings",
            yscrollcommand=cart_tree_scroll.set,
            selectmode="browse"
        )
        cart_tree_scroll.config(command=self.cart_tree.yview)

        # Configure columns
        columns={
            "Name": {"text": "Product Name", "width": 250, "anchor": "center"},
            "Price": {"text": "Unit Price", "width": 150, "anchor": "center"},
            "Quantity": {"text": "Quantity", "width": 100, "anchor": "center"},
            "Subtotal": {"text": "Subtotal", "width": 150, "anchor": "center"}
        }

        for col, config in columns.items():
            self.cart_tree.heading(col, text=config["text"])
            self.cart_tree.column(col, width=config["width"], anchor=config["anchor"])

        self.cart_tree.pack(expand=True, fill="both")
        self.populate_cart_tree()

        # Cart controls
        controls_frame=ttk.Frame(cart_frame)
        controls_frame.pack(fill="x", pady=(10, 0))

        ttk.Button(
            controls_frame,
            text="Remove Selected",
            command=lambda: self.remove_from_cart(cart_window),
            style="Warning.TButton"
        ).pack(side="left", padx=(0, 10))

        ttk.Button(
            controls_frame,
            text="Update Quantity",
            command=self.update_cart_quantity,
            style="Accent.TButton"
        ).pack(side="left", padx=(0, 10))

        # Total display
        total_frame=ttk.Frame(cart_frame)
        total_frame.pack(fill="x", pady=(10, 0))

        total=self.calculate_cart_total()

        ttk.Label(total_frame, text="Total:", font=self.button_font).pack(side="left")
        self.total_label=ttk.Label(total_frame, text=f"₹{total:.2f}", font=self.button_font)
        self.total_label.pack(side="left", padx=(5, 0))

        # Checkout button
        ttk.Button(
            cart_frame,
            text="Checkout",
            command=lambda: self.checkout(cart_window),
            style="Accent.TButton"
        ).pack(side="right", pady=(10, 0))

     # Fill items to Cart Table
    def populate_cart_tree(self):
        for item in self.cart_tree.get_children():
            self.cart_tree.delete(item)

        for product_name, quantity in self.cart.items():
            price=self.shop.Products[product_name]['Price']
            subtotal=price * quantity

            self.cart_tree.insert("", "end", values=(
                product_name,
                f"₹{price:.2f}",
                quantity,
                f"₹{subtotal:.2f}"
            ))

    #calculating total cart ammount
    def calculate_cart_total(self):
        total=0
        for product_name, quantity in self.cart.items():
            price=self.shop.Products[product_name]['Price']
            total+=price*quantity
        return total

    #removing items from cart
    def remove_from_cart(self, cart_window):
        selected_item=self.cart_tree.focus()
        if not selected_item:
            messagebox.showwarning("No Selection", "Please select an item to remove")
            return

        item_data=self.cart_tree.item(selected_item)
        product_name=item_data['values'][0]

        del self.cart[product_name]
        self.populate_cart_tree()
        self.update_total_label()

        if not self.cart:
            messagebox.showinfo("Cart Empty", "Your cart is now empty")
            cart_window.destroy()

    # update cart quantity window
    def update_cart_quantity(self):
        selected_item=self.cart_tree.focus()
        if not selected_item:
            messagebox.showwarning("No Selection", "Please select an item to update")
            return

        item_data=self.cart_tree.item(selected_item)
        product_name=item_data['values'][0]
        current_quantity=int(item_data['values'][2])

        update_window=tk.Toplevel(self.root)
        update_window.title("Update Quantity")
        update_window.geometry("300x150")

        ttk.Label(update_window, text=f"Update quantity for {product_name}").pack(pady=10)

        new_quantity=tk.IntVar(value=current_quantity)
        ttk.Spinbox(
            update_window,
            from_=1,
            to=100,
            textvariable=new_quantity,
            width=5
        ).pack(pady=5)

       # cart quantity update
        def apply_update():
            quantity=new_quantity.get()
            if quantity<=0:
                messagebox.showwarning("Invalid Quantity", "Quantity must be at least 1")
                return

            available_stock=self.shop.Products[product_name]['Stock_Quantity']
            if quantity > available_stock:
                messagebox.showwarning("Insufficient Stock",
                                    f"Only {available_stock} items available in stock")
                return

            self.cart[product_name]=quantity
            self.populate_cart_tree()
            self.update_total_label()
            update_window.destroy()

        ttk.Button(
            update_window,
            text="Update",
            command=apply_update,
            style="Accent.TButton"
        ).pack(pady=5)

    # Function to disply cart total
    def update_total_label(self):
        total=self.calculate_cart_total()
        self.total_label.config(text=f"₹{total:.2f}")

    #checkout feauture
    def checkout(self, cart_window):
        """Open billing window instead of completing purchase immediately"""
        cart_window.destroy()
        self.show_billing_window()

    # Display billing window
    def show_billing_window(self):
        billing_window=tk.Toplevel(self.root)
        billing_window.title("Billing Information")
        billing_window.geometry("800x700")

        billing_frame=ttk.Frame(billing_window, padding=15)
        billing_frame.pack(expand=True, fill="both")

        # Customer information section
        ttk.Label(billing_frame, text="Customer Details", font=self.title_font).pack(pady=(0, 10))

        info_frame=ttk.Frame(billing_frame)
        info_frame.pack(fill="x", pady=(0, 20))

        ttk.Label(info_frame, text="Name:").grid(row=0, column=0, sticky="e", padx=5, pady=5)
        self.customer_name_entry=ttk.Entry(info_frame)
        self.customer_name_entry.grid(row=0, column=1, sticky="ew", padx=5, pady=5)

        ttk.Label(info_frame, text="Phone Number:").grid(row=1, column=0, sticky="e", padx=5, pady=5)
        self.customer_phone_entry=ttk.Entry(info_frame)
        self.customer_phone_entry.grid(row=1, column=1, sticky="ew", padx=5, pady=5)

        # Order summary section
        ttk.Label(billing_frame, text="Order Summary", font=self.title_font).pack(pady=(10, 5))

        # Create a frame for the order summary with scrollbar
        summary_frame=ttk.Frame(billing_frame)
        summary_frame.pack(expand=True, fill="both")

        # Treeview with scrollbars for order items
        tree_scroll=ttk.Scrollbar(summary_frame)
        tree_scroll.pack(side="right", fill="y")

        self.summary_tree=ttk.Treeview(
            summary_frame,
            columns=("Name", "Price", "Qty", "Subtotal"),
            show="headings",
            yscrollcommand=tree_scroll.set,
            selectmode="none"  # Disable selection
        )
        tree_scroll.config(command=self.summary_tree.yview)

        # Configure columns
        columns={
            "Name": {"text": "Product Name", "width": 300, "anchor": "w"},
            "Price": {"text": "Price", "width": 100, "anchor": "e"},
            "Qty": {"text": "Qty", "width": 50, "anchor": "center"},
            "Subtotal": {"text": "Subtotal", "width": 100, "anchor": "e"}
        }

        for col, config in columns.items():
            self.summary_tree.heading(col, text=config["text"])
            self.summary_tree.column(col, width=config["width"], anchor=config["anchor"])

        self.summary_tree.pack(expand=True, fill="both")

        # Populate the order summary
        self.populate_order_summary()

        # Calculate totals
        subtotal=self.calculate_cart_total()
        tax=subtotal * (self.shop.Tax_Rate / 100)
        discount=subtotal * (self.shop.Discount_Rate / 100)
        total=subtotal + tax - discount

        # Totals display
        totals_frame=ttk.Frame(billing_frame)
        totals_frame.pack(fill="x", pady=(10, 0))

        ttk.Label(totals_frame, text="Subtotal:").grid(row=0, column=0, sticky="e")
        ttk.Label(totals_frame, text=f"₹{subtotal:.2f}").grid(row=0, column=1, sticky="e", padx=10)

        ttk.Label(totals_frame, text=f"Tax ({self.shop.Tax_Rate}%):").grid(row=1, column=0, sticky="e")
        ttk.Label(totals_frame, text=f"₹{tax:.2f}").grid(row=1, column=1, sticky="e", padx=10)

        ttk.Label(totals_frame, text=f"Discount ({self.shop.Discount_Rate}%):").grid(row=2, column=0, sticky="e")
        ttk.Label(totals_frame, text=f"-₹{discount:.2f}").grid(row=2, column=1, sticky="e", padx=10)

        ttk.Label(totals_frame, text="Total:", font=self.button_font).grid(row=3, column=0, sticky="e")
        ttk.Label(totals_frame, text=f"₹{total:.2f}", font=self.button_font).grid(row=3, column=1, sticky="e", padx=10)

        # Confirm order button
        ttk.Button(
            billing_frame,
            text="Confirm Order",
            command=lambda: self.confirm_order(billing_window),
            style="Accent.TButton"
        ).pack(side="right", pady=(15, 0))

    # Filling the summary table with order summary
    def populate_order_summary(self):
        """Populate the order summary tree with cart items"""
        for item in self.summary_tree.get_children():
            self.summary_tree.delete(item)

        for product_name, quantity in self.cart.items():
            price=self.shop.Products[product_name]['Price']
            subtotal=price * quantity

            self.summary_tree.insert("", "end", values=(
                product_name,
                f"₹{price:.2f}",
                quantity,
                f"₹{subtotal:.2f}"
            ))

    # confirm order feature
    def confirm_order(self, billing_window):
        """Process the order and save transaction"""
        customer_name=self.customer_name_entry.get().strip()
        customer_phone=self.customer_phone_entry.get().strip()

        if not customer_name:
            messagebox.showwarning("Input Error", "Please enter your name")
            return

        if not customer_phone:
            customer_phone="N/A"

        now=datetime.now()
        time_str=now.strftime("%H:%M:%S")
        date_str=now.strftime("%Y-%m-%d")

        # Generating a random Order ID
        order_id=self.shop.generate_order_id()
        for product_name, quantity in list(self.cart.items()):
            success=self.shop.buy_products(
                product_name=product_name,
                quantity=quantity,
                customer_name=customer_name,
                customer_phone=customer_phone,
                time_str=time_str,
                date_str=date_str,
                order_id=order_id
            )
            if success:
                del self.cart[product_name]
            else:
                messagebox.showerror("Error", f"Failed to process {product_name}")
                return

        billing_window.destroy()

        # Show success message with order ID
        success_window=tk.Toplevel(self.root)
        success_window.title("Order Confirmation")
        success_window.geometry("500x200")

        ttk.Label(
            success_window,
            text=f"Yay! Your order has been placed successfully!\n\nYour order ID is: {order_id}",
            font=self.title_font,
            justify="center"
        ).pack(expand=True, padx=20, pady=20)

        ttk.Button(
            success_window,
            text="Exit",
            command=success_window.destroy,
            style="Accent.TButton"
        ).pack(side="bottom", pady=(0, 20))

        # Refresh product list
        self.populate_products()


<h1>Running the Application</h1>

In [ ]:
# The GUI class will only be initialised if the current file is being run directly as the main program
if __name__ == "__main__":
    root=tk.Tk()
    app=ShoppingApp(root)
    root.mainloop()

An object of the class Shop has successfully been created !!!

Initializing Files : 
The file Products.csv already exists
The file Transactions.csv already exists
The file Admin.csv already exists

Products from Products.csv file have successfully been loaded into the data structure

Products from Transactions.csv file have successfully been loaded into the data structure

Products from Admin.csv file have successfully been loaded into the data structure

Reloading the Page : Products from Products.csv file have successfully been loaded into the data structure



2025-04-08 16:23:16.376 python[74936:1825788] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-08 16:23:16.376 python[74936:1825788] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 